In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

In [16]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback

In [5]:
df = pd.read_csv('diabetes.csv')

#View first 5 data
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df = df.drop(columns=['SkinThickness'], errors='ignore')

In [7]:
#Replace Null Value with Median

columns_to_replace = ['Glucose', 'BloodPressure', 'Insulin', 'BMI']
for col in columns_to_replace:
    df[col] = df[col].replace(0, df[col].median())

df.head()

,Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,30.5,33.6,0.627,50,1
1,1,85,66,30.5,26.6,0.351,31,0
2,8,183,64,30.5,23.3,0.672,32,1
3,1,89,66,94.0,28.1,0.167,21,0
4,0,137,40,168.0,43.1,2.288,33,1


In [8]:
predict = ['Pregnancies', 'Glucose', 'BloodPressure', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
x = df[predict]
y = df['Outcome']

In [9]:
#Standard the Dataset

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

x_scaled_df = pd.DataFrame(x_scaled, columns=predict)
x_scaled_df.head()

,Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.866045,-0.031990,-0.608201,0.167240,0.468492,1.425995
1,-0.844885,-1.205066,-0.528319,-0.608201,-0.851551,-0.365061,-0.190672
2,1.233880,2.016662,-0.693761,-0.608201,-1.331838,0.604397,-0.105584
3,-0.844885,-1.073567,-0.528319,-0.006185,-0.633239,-0.920763,-1.041549
4,-1.141852,0.504422,-2.679076,0.695378,1.549885,5.484909,-0.020496


In [10]:
#Split the data to test data and train data

x_test, x_train, y_test, y_train = train_test_split(x, y, test_size = 0.2, random_state = 1)

#Random Forest, Gradient Boosting, Decision Tree Biasa Dipake buat nge predict Medical Problems

In [18]:
#Training the model, Decision Tree

rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=1)
rf_model.fit(x_train, y_train)

y_pred = rf_model.predict(x_test)
acc_score = round(accuracy_score(y_test, y_pred), 3)

print(f"Random Forest Model Accuracy: {acc_score}")

Random Forest Model Accuracy: 0.78


In [17]:
nb_model = GaussianNB()

nb_model.fit(x_train, y_train)

nb_predictions = nb_model.predict(x_test)

nb_accuracy = accuracy_score(y_test, nb_predictions)

print(f"Naive Bayes Model Accuracy: {nb_accuracy:.2f}")

Naive Bayes Model Accuracy: 0.76


In [19]:
#Logistic Regression

model = linear_model.LogisticRegression()
model.fit(x_train, y_train)

#Test the model

acc = model.score(x_test, y_test)

print(f"Model Accuracy is {acc}")

Model Accuracy is 0.755700325732899


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
#Training the model, Logistic Regression Model

from sklearn.ensemble import GradientBoostingClassifier
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=1)
gb_model.fit(x_train, y_train)

# Predict and evaluate Gradient Boosting
y_pred_gb = gb_model.predict(x_test)
acc_score_gb = round(accuracy_score(y_test, y_pred_gb), 3)

print(f"Gradient Boosting Model Accuracy: {acc_score_gb}")

Gradient Boosting Model Accuracy: 0.757


In [13]:
tree_model = DecisionTreeClassifier()
tree_model = tree_model.fit(x_train, y_train)

y_pred = tree_model.predict(x_test)

acc_score = round(accuracy_score(y_pred, y_test), 3)

print(f"Decision Tree Accuracy {acc_score}")

Decision Tree Accuracy 0.705


In [14]:
#Neural Network

class StopAtAccuracy(Callback):
    def __init__(self, threshold):
        super().__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        acc = logs.get('accuracy')
        if acc >= self.threshold:
            print(f"\nReached {self.threshold*100:.0f}% accuracy, stopping training!")
            self.model.stop_training = True

model = Sequential([
    Dense(32, input_dim=x_train.shape[1], activation='relu'),
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
stop_at_90 = StopAtAccuracy(threshold=0.90)

history = model.fit(
    x_train, y_train,
    epochs=300,
    validation_split=0.2,
    verbose=1,
    callbacks=[stop_at_90]
)

loss, accuracy = model.evaluate(x_test, y_test)
print(f"Neural Network Model Accuracy: {accuracy:.2f}")

Epoch 1/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 190ms/step - accuracy: 0.4796 - loss: 3.0627 - val_accuracy: 0.6452 - val_loss: 1.2864
Epoch 2/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.5286 - loss: 2.2646 - val_accuracy: 0.6774 - val_loss: 0.8820
Epoch 3/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5799 - loss: 1.6057 - val_accuracy: 0.4516 - val_loss: 0.8220
Epoch 4/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.5398 - loss: 1.8394 - val_accuracy: 0.5161 - val_loss: 0.7913
Epoch 5/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.5538 - loss: 1.2966 - val_accuracy: 0.7419 - val_loss: 0.7467
Epoch 6/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.6328 - loss: 1.0847 - val_accuracy: 0.6774 - val_loss: 0.8183
Epoch 7/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.5283 - loss: 1.4800 - val_accuracy: 0.7419 - val_loss: 0.7285
Epoch 8/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5221 - loss: 1.0985 - val_accuracy: 0.6129 - val_loss